In [ ]:
import pyaudio
import numpy as np
import time
import struct

RATE = 176400
CHUNK = 20000
CHANNELS = 1
FORMAT = pyaudio.paInt16
AMPLITUDE = 4096

In [ ]:
t = np.arange(0, CHUNK/RATE, 1/RATE)

def sin_signal(freq, phi=0):
    sig = np.cos(2 * np.pi * (freq * t + phi))*AMPLITUDE
    return sig.astype(np.int16)

DTMF_1_sig = sin_signal(freq=697)
DTMF_2_sig = sin_signal(freq=770)
DTMF_3_sig = sin_signal(freq=852)
DTMF_4_sig = sin_signal(freq=941)

DTMF_A_sig = sin_signal(freq=1209)
DTMF_B_sig = sin_signal(freq=1336)
DTMF_C_sig = sin_signal(freq=1477)
DTMF_D_sig = sin_signal(freq=1633)


In [ ]:
from PIL import Image
import numpy as np

index = 0
pic = [-1]

file_path = 't.bmp'
# 打开位图文件
img = Image.open(file_path)

# 获取颜色表
color_palette = img.getpalette()

# 获取图像数据
img_data = np.array(img)

color_index_array = np.zeros_like(img_data, dtype=np.uint8)

# 遍历图像数据，将颜色映射到颜色表中的颜色索引
for i in range(16):
    mask = (img_data == i)
    color_index_array[mask] = i

for i in color_index_array:
    for j in i:
        pic.append(int(j))
    pic.append(-1)

In [ ]:
p = pyaudio.PyAudio()

def process():
    global index
    if pic[index]==-1:  out_data = DTMF_1_sig
    elif pic[index] == 0:
        out_data = DTMF_1_sig + DTMF_A_sig
    elif pic[index] == 1:
        out_data = DTMF_1_sig + DTMF_B_sig
    elif pic[index] == 2:
        out_data = DTMF_1_sig + DTMF_C_sig
    elif pic[index] == 3:
        out_data = DTMF_1_sig + DTMF_D_sig
    elif pic[index] == 4:
        out_data = DTMF_2_sig + DTMF_A_sig
    elif pic[index] == 5:
        out_data = DTMF_2_sig + DTMF_B_sig
    elif pic[index] == 6:
        out_data = DTMF_2_sig + DTMF_C_sig
    elif pic[index] == 7:
        out_data = DTMF_2_sig + DTMF_D_sig
    elif pic[index] == 8:
        out_data = DTMF_3_sig + DTMF_A_sig
    elif pic[index] == 9:
        out_data = DTMF_3_sig + DTMF_B_sig
    elif pic[index] == 10:
        out_data = DTMF_3_sig + DTMF_C_sig
    elif pic[index] == 11:
        out_data = DTMF_3_sig + DTMF_D_sig
    elif pic[index] == 12:
        out_data = DTMF_4_sig + DTMF_A_sig
    elif pic[index] == 13:
        out_data = DTMF_4_sig + DTMF_B_sig
    elif pic[index] == 14:
        out_data = DTMF_4_sig + DTMF_C_sig
    elif pic[index] == 15:
        out_data = DTMF_4_sig + DTMF_D_sig 
        
    if index <= len(pic): 
        index+=1
    else: 
        index=0
    
    return out_data

def callbackOut(in_data, frame_count, time_info, ststus_flag):
    out_data_int = process()

    out_data_byte = struct.pack('<'+CHUNK+'h', *out_data_int) 
    
    return out_data_byte, pyaudio.paContinue

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                output=True,
                frames_per_buffer=CHUNK,
                stream_callback=callbackOut)

stream.start_stream()

while index <= len(pic):
    pass    
stream.stop_stream()
print("Stream is stopped")
stream.close()

p.terminate()


In [25]:
import numpy as np
import sounddevice as sd

# 设置音频参数
sampling_rate = 44100  # 采样率
duration = 10  # 持续时间（秒）
frequency =600  # 正弦波频率

# 生成正弦波
t = np.arange(int(sampling_rate * duration)) / float(sampling_rate)
waveform = 0.5 * np.sin(2 * np.pi * frequency * t)

# 播放音频
sd.play(waveform, samplerate=sampling_rate)
sd.wait()  # 等待音频播放完成
